In [1]:
import importlib
import mc_env
import mc_job_dist
import numpy as np 
import pickle

In [2]:
importlib.reload(mc_env)
importlib.reload(mc_job_dist)

<module 'mc_job_dist' from '/home/ita/Dropbox/code/deeprm/mc_job_dist.py'>

In [3]:
lamb = "2.6"

In [4]:
with open("data/te_%s.pkl" % lamb,'rb') as f:
    seqs = pickle.load(f)

In [5]:
sz2 = 32
sz3 = 48
m_desc = [
    # machie with additional exclusive resources
    {'ex_res': [sz3, sz3, 4], 'sh_res': [1, 1], 'type': 1, 'ex_max': [sz3, sz3, 4]},
    {'ex_res': [sz2, sz2, 4], 'sh_res': [0, 0], 'type': 2, 'ex_max': [sz3, sz3, 4]},
    {'ex_res': [sz3, sz3, 0], 'sh_res': [1, 1], 'type': 1, 'ex_max': [sz3, sz3, 4]},
    {'ex_res': [sz2, sz2, 4], 'sh_res': [1, 0], 'type': 1, 'ex_max': [sz3, sz3, 4]},
    {'ex_res': [sz2, sz2, 0], 'sh_res': [0, 1], 'type': 2, 'ex_max': [sz3, sz3, 2]},
    {'ex_res': [sz2, sz2, 0], 'sh_res': [0, 0], 'type': 1, 'ex_max': [sz3, sz3, 2]}]
importlib.reload(mc_env)

e = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=True)


In [6]:
tot_jobs = [
    sz3 + sz2 + sz3 + sz2 + sz2 + sz2,
    sz3 + sz2 + sz3 + sz2 + sz2 + sz2,
    12
] # 256, 256, 12 
tot_jobs = np.array(tot_jobs)

In [7]:
tot_jobs

array([224, 224,  12])

In [8]:
rets = []
sim_lens = list(range(30, 51, 1)) * 101
for seq in seqs[:100]:
    ret = np.zeros(3)
    for idx, js in zip(sim_lens, seq):
        slen = sim_lens[idx]
        for i, j in enumerate(js):
            res_usage = j[2] * j[0] / (tot_jobs * slen)
            ret += res_usage
            if i >= slen:
                break
    rets.append(ret)
print(np.mean(rets))

13.755574313166909


In [9]:
with_boundary = True 
import numpy as np
from baselines  import *

In [27]:
bgg, edd  = 1, 100
sim_len = 50
sim_lens = list(range(30, 51, 1)) * 101
importlib.reload(mc_env)

<module 'mc_env' from '/home/ita/Dropbox/code/deeprm/mc_env.py'>

In [28]:
env = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=False)
ct, sd = [], []
tot_slowdowns = []
tot_cts = []
lls = []
tot_rews = []
for b in range(bgg, edd + 1):
    j, m = -1, -1 
    jobs, machines, allocable_jobs, allocable_machines = ob =  env.reset(seq_no=b, curr_simu_len=sim_lens[b])
    lpss = []
    ll = []
    rews = 0
    for _ in range(2000):
        j, m = mc_SJF_RR(env, allocable_jobs, allocable_machines, j, m)
        #print(len(env.job_slot.slot))
        ob, rew, done, info = env.step((j, m))
        rews += rew
        if done:
            print(env.curr_time)
            lls.append(env.curr_time)
            tot_rews.append(rews)
            break
        jobs, machines, allocable_jobs, allocable_machines = ob
    _sd = [(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]
    _ct = [(j.finish_time - j.enter_time) for i, j in env.job_record.items()]
    tot_slowdowns += _sd 
    tot_cts += _ct
    ct.append(np.mean([(j.finish_time - j.enter_time) for i, j in env.job_record.items()]))
    sd.append(np.mean([(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]))

343
256
378
415
346
360
765
347
682
400
508
376
433
754
447
513
393
357
571
909
326
383
398
711
253
286
314
225
349
345
255
305
1010
459
662
326
926
759
770
1318
427
646
346
819
330
371
308
348
845
342
353
257
754
519
621
413
1445
590
408
899
516
509
307
721
404
267
302
600
704
281
425
289
1097
278
504
318
789
340
652
407
457
709
475
238
341
284
445
357
283
308
284
682
417
226
451
511
344
607
629
364


In [30]:
print(np.mean(ct), np.mean(sd), np.mean(tot_rews))
rews1 = tot_rews
lls1 = lls

119.54833428987044 5.933625638274743 -8867.457494830665


In [33]:
env = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=False)
ct, sd = [], []
tot_slowdowns = []
tot_cts = []
lls = []
tot_rews = []
for b in range(bgg, edd + 1):
    j, m = -1, -1 
    jobs, machines, allocable_jobs, allocable_machines = ob =  env.reset(seq_no=b, curr_simu_len=sim_lens[b])
    lpss = []
    ll = []
    rews = 0
    for _ in range(2000):
        j, m = mc_RR_RR(env, allocable_jobs, allocable_machines, j, m)
        #print(len(env.job_slot.slot))
        ob, rew, done, info = env.step((j, m))
        rews += rew
        if done:
            print(env.curr_time)
            lls.append(env.curr_time)
            tot_rews.append(rews)
            break
        jobs, machines, allocable_jobs, allocable_machines = ob
    _sd = [(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]
    _ct = [(j.finish_time - j.enter_time) for i, j in env.job_record.items()]
    tot_slowdowns += _sd 
    tot_cts += _ct
    ct.append(np.mean([(j.finish_time - j.enter_time) for i, j in env.job_record.items()]))
    sd.append(np.mean([(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]))

358
254
385
504
345
352
752
368
643
392
487
363
518
819
463
595
408
338
548
1186
307
352
425
724
279
306
317
215
331
322
217
301
1097
442
746
329
1131
803
803
1534
451
718
282
923
323
345
320
325
826
335
400
259
841
585
786
379
1386
696
420
1161
621
596
308
813
381
258
296
592
625
236
368
302
1052
297
577
311
763
307
619
450
424
787
508
240
333
324
439
340
316
311
303
637
390
241
509
513
364
684
680
334


In [34]:
print(np.mean(ct), np.mean(sd), np.mean(tot_rews))
rews1 = tot_rews
lls1 = lls

157.66068784131969 18.808166303585473 -13099.527257387386


In [11]:
env = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=False)
ct, sd = [], []
tot_slowdowns = []
tot_cts = []
lls = []
tot_rews = []
for b in range(bgg, edd + 1):
    j, m = -1, -1 
    jobs, machines, allocable_jobs, allocable_machines = ob =  env.reset(seq_no=b, curr_simu_len=sim_lens[b])
    lpss = []
    ll = []
    rews = 0
    for _ in range(2000):
        j, m = mc_SJF_TETRIS(env, allocable_jobs, allocable_machines, j, m)
        #print(len(env.job_slot.slot))
        ob, rew, done, info = env.step((j, m))
        rews += rew
        if done:
            print(env.curr_time)
            lls.append(env.curr_time)
            tot_rews.append(rews)
            break
        jobs, machines, allocable_jobs, allocable_machines = ob
    _sd = [(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]
    _ct = [(j.finish_time - j.enter_time) for i, j in env.job_record.items()]
    tot_slowdowns += _sd 
    tot_cts += _ct
    ct.append(np.mean([(j.finish_time - j.enter_time) for i, j in env.job_record.items()]))
    sd.append(np.mean([(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]))

329
278
383
424
413
384
743
346
696
423
462
348
454
836
453
644
395
369
577
917
333
439
411
684
250
338
305
235
355
336
251
335
941
454
637
336
954
798
757
1321
435
712
321
831
326
358
313
352
837
359
370
314
766
547
682
420
1324
597
410
979
633
543
356
800
391
236
297
643
625
273
422
282
1054
322
551
288
755
304
638
419
438
723
448
239
336
274
445
315
278
339
303
671
393
224
433
482
386
581
657
352


In [12]:
print(np.mean(ct), np.mean(sd), np.mean(tot_rews))
rews1 = tot_rews
lls1 = lls

120.27179605683328 6.0072051303392335 -8952.711932458122


In [13]:
env = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=False)
ct, sd = [], []
tot_slowdowns = []
tot_cts = []
lls = []
tot_rews = []
for b in range(bgg, edd + 1):
    j, m = -1, -1 
    jobs, machines, allocable_jobs, allocable_machines = ob =  env.reset(seq_no=b, curr_simu_len=sim_lens[b])
    lpss = []
    ll = []
    rews = 0
    for _ in range(2000):
        j, m = mc_RR_TETRIS(env, allocable_jobs, allocable_machines, j, m, False)
        #print(len(env.job_slot.slot))
        ob, rew, done, info = env.step((j, m))
        ll.append(len(env.job_slot.slot))
        rews += rew
        if done:
            print(env.curr_time)
            lls.append(np.mean(ll))
            tot_rews.append(rews)
            break
        jobs, machines, allocable_jobs, allocable_machines = ob
    _sd = [(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]
    _ct = [(j.finish_time - j.enter_time) for i, j in env.job_record.items()]
    tot_slowdowns += _sd 
    tot_cts += _ct
    ct.append(np.mean([(j.finish_time - j.enter_time) for i, j in env.job_record.items()]))
    sd.append(np.mean([(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]))

350
254
388
499
345
381
735
323
656
441
480
322
482
978
505
685
407
360
545
1193
313
379
397
791
250
316
282
209
323
309
226
287
1076
434
749
350
1048
776
760
1443
402
731
339
949
324
343
315
321
931
364
399
288
909
677
721
358
1475
712
369
1209
634
581
309
778
400
277
296
699
616
240
409
305
1111
295
613
278
847
357
615
431
437
770
517
251
327
261
469
316
259
337
286
666
362
214
456
483
356
719
706
403


In [14]:
rews2 = tot_rews
np.mean(ct), np.mean(sd), np.mean(tot_rews)

(158.84516117261666, 18.568017359371012, -13240.064396536749)

In [15]:
import pointer_networks
import torch
importlib.reload(mc_env)
importlib.reload(pointer_networks)
#scheduler = pointer_networks.attention_net(job_dim=10, machine_dim=10, embedding_size=32)
with open('noc_def_%s/model_499.out' % lamb, 'rb') as f:
    scheduler = torch.load(f)
env = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=False)

In [16]:
tot_slowdowns = []
tot_cts = []
ct, sd = [], []
tot_rews = []
for b in range(bgg, edd + 1):
    jobs, machines, allocable_jobs, allocable_machines = ob =  env.reset(seq_no=b, curr_simu_len=sim_lens[b])
    lpss = []   
    rews = 0
    for _ in range(2000):
        jobs = torch.from_numpy(jobs).float()
        #print(jobs.shape)
        machines = torch.from_numpy(machines).float()
        a, logpas = scheduler(jobs, machines, allocable_jobs, allocable_machines, argmax=True)
        #a = (a[0],np.random.choice(allocable_machines[a[0]]))
        #print(env.job_slot.slot[0].sh_vec)
        lpss.append(np.exp(logpas.detach().numpy()))
        ob, rew, done, info = env.step(a)
        
        #print(env.curr_time)
        rews += rew
        if done:
            print(env.curr_time)
            tot_rews.append(rews)
            break
        jobs, machines, allocable_jobs, allocable_machines = ob
    _sd = [(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]
    _ct = [(j.finish_time - j.enter_time) for i, j in env.job_record.items()]
    tot_slowdowns += _sd 
    tot_cts += _ct
    ct.append(np.mean([(j.finish_time - j.enter_time) for i, j in env.job_record.items()]))
    sd.append(np.mean([(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]))

335
261
399
443
335
358
689
356
680
417
454
313
519
759
471
502
397
377
538
901
353
366
407
737
274
306
304
244
348
341
206
339
990
425
600
337
882
724
792
1300
439
636
301
852
322
323
314
329
791
344
375
270
682
547
679
419
1243
645
407
998
588
513
325
707
437
251
318
525
640
279
384
315
1014
295
505
289
743
331
544
420
422
665
472
260
401
312
428
319
269
379
306
617
373
230
394
491
374
632
619
333


In [17]:
np.mean(tot_rews)
rews2 = tot_rews

In [18]:
np.mean(ct),np.mean(sd), np.mean(ct), np.mean(sd), np.mean(lpss)#

(117.29912953360646,
 5.257638613940043,
 117.29912953360646,
 5.257638613940043,
 0.7068554)

In [19]:
import pointer_networks
import torch
importlib.reload(mc_env)
importlib.reload(pointer_networks)
#scheduler = pointer_networks.attention_net(job_dim=10, machine_dim=10, embedding_size=32)
with open('noc_def_%s_wa/model_499.out' % lamb, 'rb') as f:
    scheduler = torch.load(f)
env = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=False)

In [20]:
tot_slowdowns = []
tot_cts = []

ct, sd = [], []
tot_rews = []
for b in range(bgg, edd + 1):
    jobs, machines, allocable_jobs, allocable_machines = ob =  env.reset(seq_no=b, curr_simu_len=sim_lens[b])
    lpss = []   
    rews = 0
    for _ in range(2000):
        #print("CT ", env.curr_time)
        jobs = torch.from_numpy(jobs).float()
        #print(jobs.shape)
        machines = torch.from_numpy(machines).float()
        #print(machines.shape)
        a, logpas = scheduler(jobs, machines, allocable_jobs, allocable_machines, argmax=True)
        #a = (a[0],np.random.choice(allocable_machines[a[0]]))
        lpss.append(np.exp(logpas.detach().numpy()))
        ob, rew, done, info = env.step(a)
        #print(env.curr_time)
        rews += rew
        if done:
            print(env.curr_time)
            tot_rews.append(rews)
            break
        jobs, machines, allocable_jobs, allocable_machines = ob
    _sd = [(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]
    _ct = [(j.finish_time - j.enter_time) for i, j in env.job_record.items()]
    tot_slowdowns += _sd 
    tot_cts += _ct
    ct.append(np.mean([(j.finish_time - j.enter_time) for i, j in env.job_record.items()]))
    sd.append(np.mean([(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]))

389
252
383
462
334
317
741
339
689
387
438
328
490
877
454
496
410
365
633
858
325
362
381
677
269
345
316
223
356
333
254
338
945
422
620
329
916
765
713
1351
443
723
316
784
327
322
314
351
766
344
363
259
676
546
647
385
1371
587
400
1030
619
478
335
736
413
248
309
569
580
240
356
354
1063
277
509
297
760
311
590
428
428
632
484
232
317
330
476
327
278
309
300
674
369
265
430
524
391
639
619
344


In [21]:
np.mean(tot_rews)
rews3= tot_rews

In [22]:
np.mean(ct),np.mean(sd), np.mean(ct), np.mean(sd), np.mean(lpss)#

(117.42838685914185,
 5.705341574258227,
 117.42838685914185,
 5.705341574258227,
 0.70426786)

In [23]:
import pointer_networks
import torch
importlib.reload(mc_env)
importlib.reload(pointer_networks)
#scheduler = pointer_networks.attention_net(job_dim=10, machine_dim=10, embedding_size=32)
with open('noc_def_%s_nogr/model_499.out' % lamb, 'rb') as f:
    scheduler = torch.load(f)
env = mc_env.Env(100, seq=seqs, machine_descriptions=m_desc, with_dur=True, with_boundary=False)

In [24]:
tot_slowdowns = []
tot_cts = []

ct, sd = [], []
tot_rews = []
for b in range(bgg, edd + 1):
    jobs, machines, allocable_jobs, allocable_machines = ob =  env.reset(seq_no=b, curr_simu_len=sim_lens[b])
    lpss = []   
    rews = 0
    for _ in range(2000):
        #print("CT ", env.curr_time)
        jobs = torch.from_numpy(jobs).float()
        #print(jobs.shape)
        machines = torch.from_numpy(machines).float()
        #print(machines.shape)
        a, logpas = scheduler(jobs, machines, allocable_jobs, allocable_machines, argmax=True)
        #a = (a[0],np.random.choice(allocable_machines[a[0]]))
        lpss.append(np.exp(logpas.detach().numpy()))
        ob, rew, done, info = env.step(a)
        #print(env.curr_time)
        rews += rew
        if done:
            print(env.curr_time)
            tot_rews.append(rews)
            break
        jobs, machines, allocable_jobs, allocable_machines = ob
    _sd = [(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]
    _ct = [(j.finish_time - j.enter_time) for i, j in env.job_record.items()]
    tot_slowdowns += _sd 
    tot_cts += _ct
    ct.append(np.mean([(j.finish_time - j.enter_time) for i, j in env.job_record.items()]))
    sd.append(np.mean([(j.finish_time - j.enter_time) / j.len for i, j in env.job_record.items()]))

343
281
383
401
377
331
739
372
651
365
449
352
440
838
454
549
382
350
551
883
311
369
392
720
257
303
308
218
375
365
215
311
935
429
602
340
937
768
718
1339
455
615
322
808
349
359
308
322
740
322
421
250
750
505
698
383
1345
585
414
949
576
516
329
750
378
219
317
573
631
245
357
344
1010
293
560
275
792
310
594
423
456
621
516
219
300
280
457
321
287
336
280
647
362
274
414
483
361
579
647
357


In [25]:
np.mean(ct),np.mean(sd), np.mean(ct), np.mean(sd), np.mean(lpss)#

(117.1968728300179,
 5.479960966603728,
 117.1968728300179,
 5.479960966603728,
 0.6817531)

In [26]:
tot_rews, sd

([-3083.9903081115635,
  -2678.0817971067995,
  -5168.715962920541,
  -6394.622087182763,
  -5895.339971882258,
  -4736.195983472377,
  -7620.424203435006,
  -5295.900713589709,
  -13409.454856835444,
  -4966.482351973139,
  -7861.080767789379,
  -3482.645129699402,
  -7685.219862222867,
  -15908.383416177909,
  -10322.720447249041,
  -10005.237710833158,
  -8276.0914380876,
  -5604.293298648642,
  -10024.477800573752,
  -24486.429389146568,
  -4015.9038860983264,
  -5323.522482045908,
  -8494.548048765677,
  -13691.089644407928,
  -2676.7666370338147,
  -3393.910465023375,
  -4396.761989241029,
  -2191.031091057163,
  -4867.4650693320555,
  -5065.109234947109,
  -1474.5514237786108,
  -3815.2399053562654,
  -19918.146708311222,
  -10036.59924538936,
  -13445.455572095623,
  -5099.607743737515,
  -18908.97190369631,
  -10018.999872195396,
  -13304.724612244378,
  -39222.9802732858,
  -7824.340032002564,
  -8743.663447165067,
  -3549.44588638899,
  -16143.482753153892,
  -3923.957383699